# 01 - Exploration des Données: Classification Produits Rakuten

**Objectif:** Analyser le dataset et calculer les bounding boxes pour TOUTES les images

**⚠️ CRITIQUE:** Analyse ~85k images (15-20 min) - REQUIS pour reproduire F1=0.31

---

## 🔧 Configuration Environnement (Local + Colab)

In [ ]:
import sys, os
from pathlib import Path

IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    print("🔵 Exécution sur Google Colab")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/MyDrive/CLASSIFICATION-PRODUITS-RAKUTEN')
    os.chdir(PROJECT_ROOT)
else:
    print("🟢 Exécution en local")
    PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
    os.chdir(PROJECT_ROOT)

print(f"📁 Racine projet: {PROJECT_ROOT}")
print(f"✅ Répertoire de travail: {os.getcwd()}")

## 📦 Import Bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import json
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Seed aléatoire
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Répertoire de sortie
OUTPUT_DIR = Path('output/01_exploration')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Bibliothèques importées")

## 📊 Chargement des Données

In [ ]:
DATA_DIR = Path('data')
IMAGE_DIR = DATA_DIR / 'images'

print("Chargement des datasets...")
X_train = pd.read_csv(DATA_DIR / 'X_train_update.csv', index_col=0)
y_train = pd.read_csv(DATA_DIR / 'Y_train_CVw08PX.csv', index_col=0)
df = X_train.merge(y_train, left_index=True, right_index=True, how='inner')

print(f"✅ {len(df):,} produits chargés")
print(f"✅ {df['prdtypecode'].nunique()} classes")
df.head()

## 🔍 Inspection des Données

In [ ]:
print("Valeurs manquantes:")
print(df.isnull().sum())
print(f"\n% descriptions manquantes: {df['description'].isnull().sum() / len(df) * 100:.1f}%")

## 📈 Distribution des Classes

In [ ]:
class_counts = df['prdtypecode'].value_counts().sort_index()

# Labels des catégories (vos labels exacts)
prdtypecode_labels = {
    10: "Livre usagé",
    40: "Jeux vidéo et accessoires tech",
    50: "Accessoires de console",
    60: "Console de jeux vidéo",
    1140: "Statuette / Figurine",
    1160: "Cartes collectionnables",
    1180: "Jeux de table",
    1280: "Jouets enfants et costumes",
    1281: "Jeux de société",
    1300: "Jouets électroniques",
    1301: "Bas et chaussettes",
    1302: "Jeux extérieurs et vêtements",
    1320: "Articles pour bébé",
    1560: "Meubles intérieurs",
    1920: "Mobilier de chambre",
    1940: "Ustensiles de cuisine",
    2060: "Décoration intérieure",
    2220: "Produits pour animaux",
    2280: "Magazines et journaux",
    2403: "Livres, magazines et BD",
    2462: "Jeux d'occasion",
    2522: "Matériel de bureau",
    2582: "Mobilier de jardin",
    2583: "Équipement de piscine",
    2585: "Outillage et bricolage",
    2705: "Livre nouveau",
    2905: "Jeux pour PC"
}

df['prdtype_label'] = df['prdtypecode'].map(prdtypecode_labels)
print("✅ Labels ajoutés")
print(f"\nDistribution:")
print(class_counts)

In [ ]:
plt.figure(figsize=(16, 6))
class_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Distribution des Classes', fontsize=16, fontweight='bold')
plt.xlabel('Code Type Produit')
plt.ylabel('Nombre')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Déséquilibre: {class_counts.max() / class_counts.min():.1f}x")

## 🖼️ Analyse TOUTES les Images (~15-20 min)

**⚠️ CRITIQUE pour reproduire F1=0.31**

In [ ]:
df['imagefile'] = 'image_' + df['imageid'].astype(str) + '_product_' + df['productid'].astype(str) + '.jpg'
df['imagepath'] = df['imagefile'].apply(lambda x: str(IMAGE_DIR / 'image_train' / x))
print(f"✅ Chemins images créés")

In [ ]:
def find_inner_img_box(img):
    """Trouve la bounding box de l'image intérieure (zone non-blanche)"""
    for top in range(img.shape[0]):
        if (img[top] != 255).any(): break
    if top == img.shape[0] - 1: return None
    
    for bottom in range(img.shape[0] - 1, top, -1):
        if (img[bottom] != 255).any(): break
    
    for left in range(img.shape[1]):
        if (img[top:bottom+1, left] != 255).any(): break
    
    for right in range(img.shape[1] - 1, left, -1):
        if (img[top:bottom+1, right] != 255).any(): break
    
    return (top, left, bottom, right)

print("✅ Fonction bounding box définie")

In [ ]:
print(f"🔄 Analyse de TOUTES les {len(df):,} images...")
print(f"⏱️  Durée estimée: 15-20 minutes\n")

image_stats = []
blank_images = []
error_images = []

for idx in tqdm(df.index, desc="Traitement images"):
    try:
        img = cv2.imread(df.loc[idx, 'imagepath'], cv2.IMREAD_COLOR)
        if img is None:
            error_images.append(idx)
            continue
        
        bbox = find_inner_img_box(img)
        if bbox is None:
            blank_images.append(idx)
            continue
        
        top, left, bottom, right = bbox
        inner_w = right - left + 1
        inner_h = bottom - top + 1
        ratio = (inner_w * inner_h) / (img.shape[0] * img.shape[1])
        
        image_stats.append({
            'index': idx,
            'inner_width': inner_w,
            'inner_height': inner_h,
            'inner_ratio': ratio,
            'top': top,
            'left': left,
            'bottom': bottom,
            'right': right
        })
    except:
        error_images.append(idx)

df_img_stats = pd.DataFrame(image_stats)
print(f"\n✅ {len(df_img_stats):,} images analysées")
print(f"⚠️ {len(blank_images)} images blanches")
print(f"❌ {len(error_images)} erreurs")

In [ ]:
needs_zoom = (df_img_stats['inner_ratio'] <= 0.8).sum()
pct_zoom = needs_zoom / len(df_img_stats) * 100

print(f"\n📊 Images nécessitant zoom (≤0.8): {needs_zoom:,} ({pct_zoom:.1f}%)")
print(f"\nStatistiques ratio:")
print(df_img_stats['inner_ratio'].describe())

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_img_stats['inner_ratio'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
plt.axvline(0.8, color='red', linestyle='--', linewidth=2, label='Seuil (0.8)')
plt.title('Distribution Ratio Image Intérieure', fontsize=14, fontweight='bold')
plt.xlabel('Ratio')
plt.ylabel('Fréquence')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'inner_ratio_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 🔗 Fusion des Statistiques

In [ ]:
df_img_stats.set_index('index', inplace=True)
df = df.join(df_img_stats, how='left')

print(f"✅ Fusion complétée")
print(f"   Total lignes: {len(df):,}")
print(f"   Avec stats: {df['inner_ratio'].notna().sum():,} (100%)")

## 💾 Sauvegarde

In [ ]:
# Sauvegarder le dataframe exploré
df.to_csv(OUTPUT_DIR / 'df_explored.csv')
print(f"✅ Sauvegardé: df_explored.csv")

# Statistiques images
df_img_stats.to_csv(OUTPUT_DIR / 'image_stats.csv')
print(f"✅ Sauvegardé: image_stats.csv")

# Distribution classes
class_counts.to_csv(OUTPUT_DIR / 'class_distribution.csv')
print(f"✅ Sauvegardé: class_distribution.csv")

# Résumé JSON
summary = {
    'total_products': int(len(df)),
    'num_classes': int(df['prdtypecode'].nunique()),
    'images_analyzed': int(len(df_img_stats)),
    'images_need_zoom': int(needs_zoom),
    'pct_need_zoom': float(pct_zoom)
}

with open(OUTPUT_DIR / 'summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Sauvegardé: summary.json")

## 📋 Résumé

In [ ]:
print("=" * 70)
print("📊 RÉSUMÉ EXPLORATION")
print("=" * 70)
print(f"\n📦 Dataset:")
print(f"  • Produits: {len(df):,}")
print(f"  • Classes: {df['prdtypecode'].nunique()}")
print(f"  • Descriptions manquantes: {df['description'].isnull().sum() / len(df) * 100:.1f}%")

print(f"\n⚖️ Équilibre Classes:")
print(f"  • Plus commune: {class_counts.max():,} produits")
print(f"  • Moins commune: {class_counts.min():,} produits")
print(f"  • Ratio déséquilibre: {class_counts.max() / class_counts.min():.1f}x")

print(f"\n🖼️ Images:")
print(f"  • Analysées: {len(df_img_stats):,} / {len(df):,} (100%)")
print(f"  • Nécessitant zoom: {needs_zoom:,} ({pct_zoom:.1f}%)")
print(f"  • Images blanches: {len(blank_images)}")
print(f"  • Erreurs: {len(error_images)}")

print(f"\n✅ TOUTES les images analysées - prêt pour notebook 02!")
print("=" * 70)